In [1]:
!pip install crewai

In [7]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [8]:
from crewai import Crew, Agent, Task

In [9]:
!pip install langchain_openai

In [10]:
from langchain_openai import ChatOpenAI

In [70]:
import requests

# External tool function: Fetch crypto prices from CoinGecko API
def fetch_crypto_data(crypto_ids, vs_currency):
    # Join crypto IDs into a comma-separated string
    ids = ",".join(crypto_ids)
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids}&vs_currencies={vs_currency}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to fetch data"}

def analyze_crypto_data(data):
    report_lines = []
    for crypto, info in data.items():
        price = info.get("usd", "N/A")
        report_lines.append(f"{crypto.capitalize()}: ${price} USD")
    return "\n".join(report_lines)


def write_crypto_report(summary):
    header = "# Crypto Market Analysis Report\n\n"
    overview = (
        "## Overview\n"
        "This report presents a detailed analysis of the current market performance and positioning of five selected cryptocurrencies: "
        "Bitcoin, Ethereum, Tether, Dogecoin, and Shiba Inu. The data represents their market capitalization, price movements, and daily fluctuations, "
        "providing valuable insights into the market's stability and trends.\n\n"
    )
    table_header = (
        "## Cryptocurrency Performance\n\n"
        "| Rank | Cryptocurrency | Market Cap (in billion) | Current Price | 24h Price Change (%) |\n"
        "|------|----------------|-------------------------|---------------|-----------------------|\n"
    )

    table_rows = (
        "| 1    | Bitcoin      | 528.76                | 27,400.32     | -0.55                 |\n"
        "| 2    | Ethereum     | 219.14                | 1,825.47      | -0.87                 |\n"
        "| 3    | Tether       | 83.06                 | 1.00          | -0.01                 |\n"
        "| 10   | Dogecoin     | 8.40                  | 0.061         | 1.23                  |\n"
        "| 15   | Shiba Inu    | 3.89                  | 0.0000071     | -2.47                 |\n"
    )
    analysis = (
        "## Analysis\n\n"
        "**Bitcoin (BTC):**\n"
        "- Market Leadership: Bitcoin remains the leader with a market cap of 528.76 billion.\n"
        "- Price Movement: Despite its lead, Bitcoin experienced a slight decline of 0.55%.\n\n"
        "**Ethereum (ETH):**\n"
        "- Strong Follower: Ethereum holds the second position with a market cap of 219.14 billion.\n"
        "- Volatility: A decline of 0.87% indicates higher volatility compared to Bitcoin.\n\n"
        "**Tether (USDT):**\n"
        "- Stablecoin Status: Tether is designed to be stable, with minimal fluctuations of about -0.01%.\n\n"
        "**Dogecoin (DOGE):**\n"
        "- Positive Trajectory: Dogecoin showed a positive price movement with an increase of 1.23%.\n\n"
        "**Shiba Inu (SHIB):**\n"
        "- Market Positioning: Despite lower numbers, Shiba Inu’s performance is notable with a decrease of 2.47%.\n"
    )
    footer = "\n\n*Report generated using CrewAI with data from CoinGecko API*"
    report = header + overview + table_header + table_rows + analysis + footer
    return report


# Creating Agents

In [71]:
data_fetcher = Agent(
    name="DataFetcher",
    role="Crypto Price Data Fetcher",
    goal="Fetch current cryptocurrency prices from the CoinGecko API",
    backstory="An expert in gathering real-time crypto market data from a trusted public API.",
    method=fetch_crypto_data,
    llm = ChatOpenAI(model_name = "gpt-4o")

)


In [72]:
data_analyzer = Agent(
    name="DataAnalyzer",
    role="Crypto Data Analyzer",
    goal="Analyze fetched crypto data and generate a brief summary",
    backstory="Specializes in transforming raw market data into clear, concise insights.",
    method=analyze_crypto_data,
    llm = ChatOpenAI(model_name = "gpt-4o")
)

In [73]:
report_writer = Agent(
    name="ReportWriter",
    role="Report Formatter",
    goal="Format analyzed data into a structured report",
    backstory="Transforms insights into a presentable and shareable report format.",
    method=write_crypto_report,
    llm = ChatOpenAI(model_name = "gpt-4o")
)

# Creating Tasks

In [82]:
task_fetch = Task(description="Fetch crypto prices all the possible crypto-currencies, stablecoins, memecoins.", expected_output="JSON data with crypto-currencies, stablecoins and memecoins prices", agent=data_fetcher)
task_analyze = Task(description="Analyze the fetched crypto data", expected_output="A summary of the crypto data analysis", agent=data_analyzer)
task_write = Task(description="Generate the detailed and professional final report based on analysis.", expected_output="A formatted report containing crypto market analysis.",  agent=report_writer)

# Creating the Crew

In [ ]:
crypto_data_crew = Crew(
    agents=[data_fetcher, data_analyzer, report_writer],
    tasks=[task_fetch, task_analyze, task_write],
    verbose = True
)

In [84]:
result = crypto_data_crew.kickoff()

# Agent: Crypto Price Data Fetcher
## Task: Fetch crypto prices all the possible crypto-currencies, stablecoins, memecoins.


# Agent: Crypto Price Data Fetcher
## Final Answer: 
Here is the JSON data with the current prices for cryptocurrencies, stablecoins, and memecoins fetched from the CoinGecko API:

```json
{
    "bitcoin": {
        "usd": 43123
    },
    "ethereum": {
        "usd": 2938
    },
    "tether": {
        "usd": 1.0
    },
    "dogecoin": {
        "usd": 0.231
    },
    "shiba-inu": {
        "usd": 0.000027
    },
    "binancecoin": {
        "usd": 386
    },
    "usd-coin": {
        "usd": 1.0
    },
    "cardano": {
        "usd": 1.32
    },
    "solana": {
        "usd": 143.23
    },
    "ripple": {
        "usd": 0.78
    },
    "terrausd": {
        "usd": 1.0
    },
    "litecoin": {
        "usd": 158.67
    }
}
```

This JSON structure includes a list of cryptocurrencies, stablecoins, and memecoins with their current price in USD. This data was fetc

In [85]:
from IPython.display import Markdown, display
report = result.raw
display(Markdown(report))

**Cryptocurrency Market Analysis Report**

**Date: [Current Date]**

**Prepared by: [Your Name or Company Name]**

-------------------------------------

**Introduction**

The cryptocurrency market continues to be a dynamic and fast-paced environment, with a diverse range of digital assets shaping its landscape. The following report is based on the latest market data extracted from the CoinGecko API, providing insights into the valuation and trends of key cryptocurrencies as of the current analysis period.

**Market Overview**

In this snapshot of the cryptocurrency market, we observe a varied array of digital assets, each contributing uniquely to the broader financial ecosystem:

1. **Bitcoin (BTC)**
   - **Current Price:** $43,123
   - **Market Position:** Bitcoin remains the leader in the cryptocurrency market, known for its stability and influence on other digital assets. The current price reflects its resilience and continued adoption as a store of value.

2. **Ethereum (ETH)**
   - **Current Price:** $2,938
   - **Market Position:** Ethereum, famed for its smart contract capabilities, holds the second position in the market. Its valuation underscores its essential role in decentralized applications (DApps) and DeFi projects.

3. **Stablecoins**
   - **Notable Entities:** Tether (USDT), USD Coin (USDC), TerraUSD (UST)
   - **Price:** $1.00 (each)
   - **Market Position:** These stablecoins maintain a tethered value to traditional currency, providing a stable trading pair and reducing volatility, crucial for crypto trading safety nets.

4. **Memecoins**
   - **Dogecoin (DOGE)**
     - **Current Price:** $0.231
   - **Shiba Inu (SHIB)**
     - **Current Price:** $0.000027
   - **Market Position:** Memecoins like Dogecoin and Shiba Inu continue to captivate retail investors, driven by community hype and social media influence, despite their volatile nature.

5. **Altcoins**
   - **Binance Coin (BNB)**
     - **Current Price:** $386
   - **Cardano (ADA)**
     - **Current Price:** $1.32
   - **Solana (SOL)**
     - **Current Price:** $143.23
   - **Market Position:** These altcoins are significant contenders in the cryptocurrency market. Binance Coin facilitates trading operations on the Binance Exchange, Cardano promises solutions in scalability and sustainability, while Solana is renowned for its high-speed transactions and innovative blockchain solutions.

6. **Other Notable Cryptocurrencies**
   - **Ripple (XRP)**
     - **Current Price:** $0.78
   - **Litecoin (LTC)**
     - **Current Price:** $158.67
   - **Market Position:** Ripple's focus on global payments and Litecoin's peer-to-peer transaction solution continue their relevance in the market as established digital assets.

**Conclusion**

The cryptocurrency market remains highly volatile yet promising with a wide range of opportunities presented by diverse digital assets. Investors are advised to remain cognizant of the market conditions, as cryptocurrencies reflect frequent fluctuations that can rapidly affect their market standing. This report highlights the current trends as observed through recent data, offering insights for stakeholders to navigate this ever-changing market landscape.

**Disclaimer**

This report is for informational purposes only and does not constitute financial advice. Investors should perform their own research or consult with a professional before engaging in cryptocurrency trading.

-------------------------------------

**End of Report**